# Convert Wingtra flight plan to KML

In [53]:
# Author: Farid Javadnejad
# Date: 2025-02-18
#
# DESCRIPTION:
# This script converts a Wingtra UAS flight plan from a .flightplan file (i.e., JSON) to a KML file, 
# extracting the survey area and adding a Placemark for the takeoff/landing location.
#
# DISCLAIMER:
# This script was developed with the assistance of AI tools for debugging, reviewing, and testing.
# ---------------------------------------------------------------------------------------------------

In [54]:
#pip install simplekml fastkml lxml
import json
import os
import simplekml

In [83]:
import os
import simplekml

def read_flightplan(file_path):
    with open(file_path, "r") as file:
        return json.load(file)

def extract_survey_area(flightplan):
    for item in flightplan.get("flightPlan", {}).get("items", []):
        if item.get("complexItemType") == "area" and "polygon" in item:
            return item["polygon"]
    return None

def extract_corridors(flightplan):
    corridors = []
    for item in flightplan.get("flightPlan", {}).get("items", []):
        if item.get("complexItemType") == "corridor" and "corridor" in item:
            polyline = item["corridor"].get("polyline", [])
            if polyline:
                corridors.append(polyline)
    return corridors

def convert_to_kml_coordinates(coords):
    try:
        return [(point[1], point[0]) for point in coords]
    except IndexError:
        print("Invalid coordinate format.")
        return None

def create_kml_file(kml_coordinates, corridors, takeoff_landing, file_path):
    kml = simplekml.Kml()
    
    # Add area polygon
    if kml_coordinates:
        pol = kml.newpolygon(name="Flight Area", outerboundaryis=kml_coordinates)
        pol.style.polystyle.color = simplekml.Color.changealpha("4C", simplekml.Color.rgb(0, 0, 0))  # 30% opacity
        pol.style.linestyle.color = simplekml.Color.rgb(255, 255, 255)
        pol.style.linestyle.width = 1.0
    
    # Add corridors as polylines
    for i, corridor in enumerate(corridors):
        kml_coords = convert_to_kml_coordinates(corridor)
        if kml_coords:
            line = kml.newlinestring(name=f"Corridor {i+1}", coords=kml_coords)
            line.style.linestyle.color = simplekml.Color.rgb(255, 0, 0)  # Red color
            line.style.linestyle.width = 2.0
    
    # Add takeoff/landing point
    if takeoff_landing:
        kml.newpoint(name="Takeoff/Landing", coords=[(takeoff_landing[1], takeoff_landing[0], takeoff_landing[2])])
    
    kml_filename = os.path.splitext(file_path)[0] + ".kml"
    kml.save(kml_filename)
    print(f"KML file saved: {kml_filename}")

def convert_flightplan_to_kml(file_path):
    flightplan = read_flightplan(file_path)
    survey_area = extract_survey_area(flightplan)
    corridors = extract_corridors(flightplan)

    kml_coordinates = convert_to_kml_coordinates(survey_area) if survey_area else None
    takeoff_landing = flightplan.get("flightPlan", {}).get("items", [])[0].get("coordinate", [])
    create_kml_file(kml_coordinates, corridors, takeoff_landing, file_path)

In [84]:
read_folder = "C:/Farid/Projects/20250225 - NM 128 P1-P2 Additional Survey/Flight Plans/P1-A"
read_file = "NM 128 P1-A Corridor LiDAR.flightplan"
file_path = str(read_folder) + "/" + str(read_file)

convert_flightplan_to_kml(file_path)

KML file saved: C:/Farid/Projects/20250225 - NM 128 P1-P2 Additional Survey/Flight Plans/P1-A/NM 128 P1-A Corridor LiDAR.kml
